## Problem 6.3: Analysis of FRAP data (40 pts)

In [homework set 4](hw4.html#Problem-4.1:-Analysis-of-FRAP-data-(40-pts)), we began analyzing a FRAP experiment by Nate Goehring and corworkers. You performed image analysis to obtain the mean fluorescence of the bleach spot versus time. In this problem, you will use those data to obtain estimates for the diffusion coefficient $D$ and chemical rate constant $k_\mathrm{off}$ for the PH-PLCd1/PIP2 complex.

As a reminder, we are taking a simplified approach, but there is more sophisticated analysis we can do to get better estimates for the phenomenological coefficients. These are discussed in the [Goehring, et al. paper](http://dx.doi.org/10.1016/j.bpj.2010.08.033). Instead, we will use the the mean fluorescence of the bleached region, $I(t)$ to perform our analysis. As derived in their paper, 

\begin{align}
I_\mathrm{norm}(t) \equiv I(t)/I_0 &= 
f_f\left(1 - f_b\,\frac{4 \mathrm{e}^{-k_\mathrm{off}t}}{d_x d_y}\,\psi_x(t)\,\psi_y(t)\right),\\[1mm]
\text{where } \psi_i(t) &= \frac{d_i}{2}\,\mathrm{erf}\left(\frac{d_i}{\sqrt{4Dt}}\right)
-\sqrt{\frac{D t}{\pi}}\left(1 - \mathrm{e}^{-d_i^2/4Dt}\right),
\end{align}

where $d_x$ and $d_y$ are the extent of the photobleached box in the $x$- and $y$-directions, $f_b$ is the fraction of fluorophores that were bleached, $f_f$ is the fraction of total fluorescent species left after photobleaching, and $\mathrm{erf}(x)$ is the [error function](http://en.wikipedia.org/wiki/Error_function). Here, $I_0$ is the mean fluorescence in the bleach spot before bleaching. Note that this function is defined such that the photobleaching event occurs at time $t = 0$.

Your task in this problem is to develop a generative model and then to find estimates for the parameters of the model. We will revisit this problem again later in the course and build a hierarchical model. For this problem, consider each of the eight trials separately and use optimization to find the MAP parameter values for each trial and make a Gaussian approximation of the posterior to give an approximate 95% credible region.

You should have acquired a data set of mean fluorescence versus time, and you should use that for your analysis. If you do not have that data set, you can download those generated in the [solutions](http://bebi103.caltech.edu/2018_protected/hw_solutions/hw4_solutions.html) [here](http://bebi103.caltech.edu/2018_protected/hw_solutions/hw_4.1_frap_image_processing_results.csv).

In [3]:
import numpy as np
import pandas as pd
import scipy.special
import scipy.stats as st

import bebi103

import altair as alt
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

Parameters to estimate: $I_0$, $f_b$, $f_f$, $k_\mathrm{off}$, $D$, $e_i$

$I_0$ = beta(10,.1)
This is estimated as the maximum of the dataset, which is likely very near the maximum fluorescence in the experiment, which we will estimate as 1, so we want a distribution over (0,1) that is clustered around 1 but does not go over. We will use a beta distribution with alpha = 10 and beta =.1


$f_f$ = HalfNorm(0, .1)
This depends on how we normalize, if we normalize such that the min is 0, this should be 0. If we only normalize by the max, this will be min/max. Assuming that we are normalizing such that the min is 0, let's use a halfnormal distribution with a rapid decline

$f_b$ = 1 - $f_f$



$D$ = LogNorm(ln(1.7), .75)


$k_\mathrm{off}$ = LogNorm(ln(.12), .75)






In [10]:
phi = np.linspace(0, 20, 200)
p = bokeh.plotting.figure(width=300, height=200, 
                          x_axis_label='D (μm2/s)', 
                          y_axis_label='g(φ)')
p.line(phi, st.lognorm.pdf(phi, .75, loc=0, scale=1.7), line_width=2)
bokeh.io.show(p)

In [12]:
phi = np.linspace(0, 2, 200)
p = bokeh.plotting.figure(width=300, height=200, 
                          x_axis_label='koff (1/s)', 
                          y_axis_label='g(φ)')
p.line(phi, st.lognorm.pdf(phi, .75, loc=0, scale=.12), line_width=2)
bokeh.io.show(p)

In [21]:
sigma = np.linspace(0, 2, 200)
p = bokeh.plotting.figure(width=300, height=200, 
                          x_axis_label='σ (µm)', 
                          y_axis_label='g(σ)')
p.line(sigma, st.norm.pdf(sigma, 1, .1), line_width=2)
bokeh.io.show(p)